In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import skimage, cv2, scipy
from skimage import io, transform
from tqdm import tqdm
from copy import copy
%matplotlib inline

In [2]:
joints=pd.read_csv("./dataset(basic).csv",header=None).as_matrix()

In [3]:
img_path=list(path for path in joints[:,0])
joint_coors=list(coors for coors in joints[:,1:29])
joint_is_valid=list(is_valid for is_valid in joints[:,29:])

In [4]:
def process(img_path, joint_coors, joint_is_valid, re_img_size=(256,256), is_valid=False):
    img_set=np.zeros([len(img_path),re_img_size[0],re_img_size[1],3],dtype=np.float)
    coor_set=np.array(joint_coors).reshape(len(joint_coors),14,2)
    
    with tqdm(total=len(img_path)) as pbar:
        pbar.set_description("Processing Images & Coordinates")
        for i, path in enumerate(img_path):
            img=skimage.io.imread(path)
            img_set[i]=skimage.transform.resize(img,(re_img_size[0],re_img_size[1],3))

            for j in range(14):
                if is_valid and bool(joint_is_valid[i][j]): coor_set[i][j] = [-1,-1]

                if coor_set[i][j][0]==-1: pass
                else: coor_set[i][j][0]=coor_set[i][j][0]*(re_img_size[0]/img.shape[1])

                if coor_set[i][j][1]==-1: pass
                else: coor_set[i][j][1]=coor_set[i][j][1]*(re_img_size[1]/img.shape[0])
            pbar.update(1)
    return img_set, coor_set

In [5]:
def rotation(images, joints, joint_is_valid):
    thetas = np.deg2rad((-30,-20,-10,10,20,30))
    rotated_img = np.zeros([images.shape[0]*len(thetas), images.shape[1],images.shape[2],3])
    rotated_coor = np.zeros([joints.shape[0]*len(thetas), joints.shape[1],2])
    rotated_valid = joint_is_valid
    
    with tqdm(total=len(images)*len(thetas)) as pbar:
        pbar.set_description("Rotating Images & Coordinates")
        for i, img in enumerate(images):
            for j, theta in enumerate(thetas):
                img_rotated = scipy.ndimage.rotate(img, theta)
                org_center = (np.array(img.shape[:2][::-1])-1)/2
                rotated_center = (np.array(img_rotated.shape[:2][::-1])-1)/2

                coor_list = list(np.array(coor)+org_center for coor in joints[i])
                rotated_coor_list = list((lambda x : (x[0]*np.cos(theta) + x[1]*np.sin(theta) + rotated_center[0],
                                                      -x[0]*np.sin(theta) + x[1]*np.cos(theta) + rotated_center[1])
                                         )(coor) for coor in coor_list)

                img_rotated = skimage.transform.resize(img_rotated, (256,256,3))

                rotated_img[(i*len(thetas))+j]=img_rotated
                rotated_coor[(i*len(thetas))+j]=np.array(rotated_coor_list)*(256/img_rotated.shape[0])
                pbar.update(1)
    for i in range(len(thetas)-1):
        rotated_valid = np.concatenate((rotated_valid, joint_is_valid), axis=0)
            
    return {'images':rotated_img,'joints':rotated_coor,'valid':rotated_valid}

In [6]:
def mirroring(images, joints, joint_is_valid):
    mirrored_img = np.zeros([images.shape[0], images.shape[1],images.shape[2],3])
    mirrored_coor = np.zeros([joints.shape[0], joints.shape[1],2])
    
    with tqdm(total=len(images)) as pbar:
        pbar.set_description("Mirroring Images & Coordinates")
        for i, img in enumerate(images):
            mirrored_img[i] = np.fliplr(img)
            for j, joint in enumerate(joints[i]):
                mirrored_coor[i][j][1] = joint[1]
                if joint[0] > (img.shape[0]/2):
                    mirrored_coor[i][j][0] = (lambda x : x-2*(x-(img.shape[0]/2)))(joint[0])
                else:
                    mirrored_coor[i][j][0] = (lambda x : x+2*((img.shape[0]/2)-x))(joint[0])
            pbar.update(1)
    return {'images':mirrored_img,'joints':mirrored_coor,'valid':joint_is_valid}

In [7]:
def shuffling(images, joints, joint_is_valid):
    shuffled_img = np.zeros([images.shape[0], images.shape[1],images.shape[2],3])
    shuffled_coor = np.zeros([joints.shape[0], joints.shape[1],2])
    shuffled_valid = np.zeros([len(joint_is_valid),14])
        
    indices=list(range(len(picpic)))
    np.random.shuffle(indices)
        
    for i, idx in enumerate(indices):
        shuffled_img[i] = images[idx]
        shuffled_coor[i] = joints[idx]
        shuffled_valid[i] = joint_is_valid[idx]
            
    return {'images': shuffled_img,'joints':shuffled_coor,'valid':shuffled_valid}

In [8]:
def markJoints(img, joints, joint_is_valid, is_valid=False):  
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(14):
        x, y = map(int,joints[i])
        if x!=-1: 
            if is_valid and bool(joint_is_valid[i]): continue
            else:
                rr, cc = skimage.draw.circle(y, x, 5) 
                skimage.draw.set_color(img, (rr, cc), (1,0,0)) 
                cv2.putText(img, str(i+1), (x,y), font, 0.5, (0.5,0.5,0.5), 2, cv2.LINE_AA)
    return img

In [9]:
picpic, coco = process(img_path, joint_coors, joint_is_valid)

print(picpic.shape)

Processing Images & Coordinates:   0%|          | 0/960 [00:00<?, ?it/s]/home/hahnz/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Processing Images & Coordinates: 100%|██████████| 960/960 [00:22<00:00, 43.34it/s] 

(960, 256, 256, 3)


In [10]:
rotated = rotation(copy(picpic), copy(coco), copy(joint_is_valid))

picpic = np.concatenate((picpic, rotated['images']), axis=0)
coco = np.concatenate((coco, rotated['joints']), axis=0)
joint_is_valid = np.concatenate((joint_is_valid, rotated['valid']), axis=0)

print(picpic.shape)

Rotating Images & Coordinates:   0%|          | 0/5760 [00:00<?, ?it/s]/home/hahnz/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Rotating Images & Coordinates: 100%|██████████| 5760/5760 [02:40<00:00, 35.86it/s]


(6720, 256, 256, 3)


In [11]:
fliplred = mirroring(copy(picpic), copy(coco), copy(joint_is_valid))

picpic = np.concatenate((picpic, fliplred['images']), axis=0)
coco = np.concatenate((coco, fliplred['joints']), axis=0)
joint_is_valid = np.concatenate((joint_is_valid, fliplred['valid']), axis=0)

print(picpic.shape)

Mirroring Images & Coordinates: 100%|██████████| 6720/6720 [00:05<00:00, 1181.03it/s]


(13440, 256, 256, 3)


In [12]:
shuffled = shuffling(copy(picpic), copy(coco), copy(joint_is_valid))

picpic = shuffled['images']
coco = shuffled['joints']
joint_is_valid = shuffled['valid']

In [13]:
print(picpic.shape)

(13440, 256, 256, 3)
